# Sampling and Spatial Resolution
stough 202-

One of the core properties of any digital image is its *resolution*. But this is a very loaded word in that it is used in a variety of ways for different purposes. Very generally, the light from the [external world](https://en.wikipedia.org/wiki/Solipsism) comes at our digital camera from a virtually uncountably infinite number of directions, with each direction specifying a "color" among a virtually uncountably infinite number of color distributions and intensities. Resolution broadly refers to the finite sampling of this infinite world. In the case of *spatial resolution* that is simply the number of *pixels* or "picture elements" that our camera can possibly obtain (see "internal image plane" below). Beyond spatial resolution, there is also [color resolution](./color_quantization.ipynb), which we get to elsewhere. 



<img src="../dip_figs/camera_diagram.png" style="height:300px"/> <img src="../dip_figs/image_acqusition.jpeg" style="height:300px"/>

In this notebook we're going to look at the spatial resolution of a digital image.

## Imports
We'll use the `rescale` and method in the powerful [`scikit-image`](https://scikit-image.org/) module in order to change image resolution. We'll also be using some widgets to put together an [interactive visualization](../NumpyAndVisualization/interactive_vis.ipynb).

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
from skimage.transform import rescale
from ipywidgets import VBox, IntSlider, AppLayout

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

import matrix_utils

In [ ]:
I = plt.imread('../dip_pics/grandCanyon.jpg')
matrix_utils.arr_info(I)

As you can see the image matrix `I` is 2160 rows by 3840 columns, with three color channels (red, green, blue) in the integer range $[0,255]$. By the way this is literally a "4K" image.

- "4K" refers to the approximate number of columns, or the left-right sampling of the image. At an aspect ratio of 16(width, columns):9(height, rows), a "1080" high definition image or screen is 1920 pixels across, about 2K. So a 4K is a 2x2 grid of such 2K screens. It has 4 times the number of pixels in a 2K (or 1080). It's more [complicated](https://www.cnet.com/news/4k-1080p-2k-uhd-8k-tv-resolutions-explained/) than that, but not by much.

If we think of an image as a grid of pixels, where the center of each pixel represents that pixel's coordinates, then changing the resolution of an image involves resampling the space between those pixel coordinates. We'll avoid most of the math here by relying on `skimage`, but take a very simple example: If we wanted to halve the number of pixels both in width and height without cropping, we could just take the average of every 2x2 pixels. Similarly, if we wanted to double the number of pixels in both directions, we could simply make a 2x2 grid of copies of each pixel. 

We'll start by doing this halving of resolution. We'll first ask [`rescale`](https://scikit-image.org/docs/dev/api/skimage.transform.html?highlight=rescale#rescale) to give us an image 1/4 the number of pixels. But then we'll *upsample* that image by four. The `order` and `anti-aliasing` parameters are used to define the level of "smoothing" that takes place, i.e. how many neighboring pixels to consider when deciding the color of the new sample coordinate.

In [ ]:
# The image is so large, take a tiny piece of it to test the order, anti-aliasing parameters.
# I = I[800:930, 1900:2050, :]

In [ ]:
sI = rescale(I, .25,
             order=0, # use bilinear sampling
             anti_aliasing=False,
             multichannel=True)

# rescale back up
reI = rescale(sI, 4,
              order=0, # use nearest neighbor sampling
              anti_aliasing=False,
              multichannel=True)

f, ax = plt.subplots(1,2, sharex=True, sharey=True,
                     figsize=(12, 3))

ax[0].imshow(I, interpolation=None)
ax[0].set_title('Original')

ax[1].imshow(reI, interpolation=None)
ax[1].set_title('Quarter Res')

# [a.axes.get_xaxis().set_visible(False) for a in ax]
# [a.axes.get_yaxis().set_visible(False) for a in ax]
plt.tight_layout()

If you're displaying this notebook in anything less than 8K full screen it's likely that the original and quarter-resolution images look identical. Use the zoom function to look more closely at particular areas to observe the difference. As you zoom in you should start to see the pixels that make up both images, and you will see the difference in spatial resolution. In this display, the reason I upsample the just downsampled image is so that the display could `sharex` and `sharey`.

Below is an interactive visualization that allows you to select the degree of downsampling in the right-hand image. First, we'll define a quick function that does the resampling.

In [ ]:
def downandup(I, factor):
    # downscale
    sI = rescale(I, 1/factor,
                 order=0, # use nearest neighbor
                 anti_aliasing=False,
                 multichannel=True)

    # rescale back up
    reI = rescale(sI, factor,
                  order=0, # use nearest neighbor sampling
                  anti_aliasing=False,
                  multichannel=True)
    return reI

In [ ]:
plt.ioff()

# Create a slider object
slider = IntSlider(
    orientation='horizontal',
    description='Exponent:',
    value=0,
    min=0,
    max=10
)

slider.layout.margin = '0px 30% 0px 30%'
slider.layout.width = '40%'


f, ax = plt.subplots(1,2, figsize=(10,3), sharex=True, sharey=True)
f.canvas.header_visible = False
ax[0].imshow(I, interpolation=None)
ax[0].set_title('Original')

rdisp = ax[1].imshow(I, interpolation=None)
rtext = ax[1].set_title(f'Factor: {2**slider.value:d}')


# A function that will be called whenever the slider changes.
def update(change):
    Ire = downandup(I, factor=2**slider.value)
    rdisp.set_data(Ire)
    rtext.set_text(f'Factor: {2**slider.value:d}')
    f.canvas.draw()
    f.canvas.flush_events()

# Connecting the slider object to the update function above.
# This is event-handling.
slider.observe(update, names='value')

plt.tight_layout()
# Creates an application interface with the various 
# pieces we already instantiated inside of it. 
AppLayout(
    center=f.canvas,
    footer=slider,
    pane_heights=[0, 6, 1]
)

Note that I'm using the `slider.value` as an exponent to give a little more effect. Think of the printed "Factor" ($2^{slider.value}$) as the number of pixels to combine in each dimension. So at an exponent of 3, we're scaling the image by 8 in each dimension; i.e., each pixel in the downsampled image represents 64 pixels in the original.